<a href="https://colab.research.google.com/github/asokraju/Rienforcement-Learning/blob/master/Q_learning_Batch_method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

#from pyvirtualdisplay import Display
#Display().start()

import gym
#from IPython import display
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import random

import sklearn.pipeline
import sklearn.preprocessing
 

from sklearn.linear_model import SGDRegressor
from sklearn.kernel_approximation import RBFSampler
from collections import namedtuple
import itertools
import sys

In [0]:
env = gym.envs.make("CartPole-v1")

In [0]:

def Samples_gen():
  """generates iid samples"""
  env.reset()
  state,_,_, _ =env.step(random.choice([0,1]))
  return state
observation_examples = np.array([Samples_gen() for x in range(10000)])
scaler = sklearn.preprocessing.StandardScaler()
scaler.fit(observation_examples)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [0]:
featurizer = sklearn.pipeline.FeatureUnion([
        ("rbf1", RBFSampler(gamma=5.0, n_components=100)),
        ("rbf2", RBFSampler(gamma=2.0, n_components=100)),
        ("rbf3", RBFSampler(gamma=1.0, n_components=100)),
        ("rbf4", RBFSampler(gamma=0.5, n_components=100))
        ])
featurizer.fit(scaler.transform(observation_examples))

FeatureUnion(n_jobs=None,
             transformer_list=[('rbf1',
                                RBFSampler(gamma=5.0, n_components=100,
                                           random_state=None)),
                               ('rbf2',
                                RBFSampler(gamma=2.0, n_components=100,
                                           random_state=None)),
                               ('rbf3',
                                RBFSampler(gamma=1.0, n_components=100,
                                           random_state=None)),
                               ('rbf4',
                                RBFSampler(gamma=0.5, n_components=100,
                                           random_state=None))],
             transformer_weights=None, verbose=False)

In [0]:
class Estimator():
    """
    Value Function approximator. 
    """
    
    def __init__(self):
        # We create a separate model for each action in the environment's
        # action space. Alternatively we could somehow encode the action
        # into the features, but this way it's easier to code up.
        self.models = []
        for _ in range(env.action_space.n):
            model = SGDRegressor(learning_rate="constant")
            # We need to call partial_fit once to initialize the model
            # or we get a NotFittedError when trying to make a prediction
            # This is quite hacky.
            model.partial_fit(self.featurize_state([env.reset()]), [0])
            model.partial_fit(self.featurize_state([env.reset()]), [1])
            self.models.append(model)
    
    def featurize_state(self, state):
        """
        Returns the featurized representation for a state.
        """
        scaled = scaler.transform(state)
        featurized = featurizer.transform(scaled)
        return featurized
    
    def predict(self, s, a=None):
        """
        Makes value function predictions.
        
        Args:
            s: state to make a prediction for
            a: (Optional) action to make a prediction for
            
        Returns
            If an action a is given this returns a single number as the prediction.
            If no action is given this returns a vector or predictions for all actions
            in the environment where pred[i] is the prediction for action i.
            
        """
                
        # TODO: Implement this!
        if a is not None:
            prediction = self.models[a].predict(self.featurize_state([s]))
            return prediction[0]
        
        else:
            predictions = np.array([self.models[i].predict(self.featurize_state([s])) for i in range(env.action_space.n)])
            return predictions.reshape(-1)
    def update(self, s, a, y):
        """
        Updates the estimator parameters for a given state and action towards
        the target y.
        """
        # TODO: Implement this!

        self.models[a].partial_fit(self.featurize_state(s), y)

In [0]:

def make_epsilon_greedy_policy(estimator, epsilon, nA):
    """
    Creates an epsilon-greedy policy based on a given Q-function approximator and epsilon.
    
    Args:
        estimator: An estimator that returns q values for a given state
        epsilon: The probability to select a random action . float between 0 and 1.
        nA: Number of actions in the environment.
    
    Returns:
        A function that takes the observation as an argument and returns
        the probabilities for each action in the form of a numpy array of length nA.
    
    """
    def policy_fn(observation):
        A = np.ones(nA, dtype=float) * epsilon / nA
        q_values = estimator.predict(observation)
        best_action = np.argmax(q_values)
        A[best_action] += (1.0 - epsilon)
        return A
    return policy_fn

In [0]:
def q_learning(env, estimator, num_episodes, discount_factor=1.0, epsilon=0.1, epsilon_decay=1.0):    
  """
    Q-Learning algorithm for off-policy TD control using Function Approximation.
    Finds the optimal greedy policy while following an epsilon-greedy policy.
    
    Args:
        env: OpenAI environment.
        estimator: Action-Value function estimator
        num_episodes: Number of episodes to run for.
        discount_factor: Gamma discount factor.
        epsilon: Chance the sample a random action. Float betwen 0 and 1.
        epsilon_decay: Each episode, epsilon is decayed by this factor
    
    Returns:
        An EpisodeStats object with two numpy arrays for episode_lengths and episode_rewards.
    """
  nA = env.action_space.n
  EpisodeStats = namedtuple("Statistics", ["episode_lengths", "episode_rewards"])
  # Keeps track of useful statistics
  stats = EpisodeStats(episode_lengths=np.zeros(num_episodes),
                      episode_rewards=np.zeros(num_episodes)
                      ) 
  for i_episode in range(num_episodes):
      
      # The policy we're following
      policy = make_epsilon_greedy_policy(
          estimator, epsilon * epsilon_decay**i_episode, nA)
      
      # Print out which episode we're on, useful for debugging.
      # Also print reward for last episode
      last_reward = stats.episode_rewards[i_episode - 1]
      print("\rEpisode {}/{} ({})".format(i_episode + 1, num_episodes, last_reward))
      sys.stdout.flush()
      
      # TODO: Implement this!
      states_1 = []
      targets_1 = []
      actions_1 = []
      states_0 = []
      targets_0 = []
      actions_0 = []
      state = env.reset()
      #states.append(state)
      for t in itertools.count():
          
          # sample the action from the epsilon greedy policy
          action = np.random.choice(nA, p=policy(state))
          
          # Perform the action -> Get the reward and observe the next state
          new_state, reward, terminated, _ = env.step(action)
          new_action = np.random.choice(nA, p=policy(new_state))
                      
          stats.episode_rewards[i_episode] += reward
          stats.episode_lengths[i_episode] = t
          
          q_values_new_state = estimator.predict(new_state)
          # value that we should have got
          # The Q-learning target policy is a greedy one, hence the `max`
          td_target = reward + discount_factor * np.max(q_values_new_state)
          #estimator.update(state, action, td_target)            
          
          # update current state
          if action == 1:
            states_1.append(list(state))
            actions_1.append(action)
            targets_1.append(td_target)
          else:
            states_0.append(list(state))
            actions_0.append(action)
            targets_0.append(td_target)
          if terminated:
              break
          state = new_state
      #print(targets_1)
      #print(targets_0)
      if len(states_1)>0:
        estimator.update(states_1, 1, np.array(targets_1).ravel())
      if len(states_0)>0:
        estimator.update(np.array(states_0), 0, np.array(targets_0).ravel())
  return stats
    
estimator = Estimator()    

In [0]:
stats = q_learning(env, estimator, 100*1000, epsilon=0.99, epsilon_decay=0.999)


Episode 1/100000 (0.0)
Episode 2/100000 (31.0)
Episode 3/100000 (28.0)
Episode 4/100000 (12.0)
Episode 5/100000 (12.0)
Episode 6/100000 (23.0)
Episode 7/100000 (11.0)
Episode 8/100000 (59.0)
Episode 9/100000 (19.0)
Episode 10/100000 (17.0)
Episode 11/100000 (25.0)
Episode 12/100000 (31.0)
Episode 13/100000 (26.0)
Episode 14/100000 (11.0)
Episode 15/100000 (10.0)
Episode 16/100000 (71.0)
Episode 17/100000 (11.0)
Episode 18/100000 (16.0)
Episode 19/100000 (20.0)
Episode 20/100000 (22.0)
Episode 21/100000 (16.0)
Episode 22/100000 (31.0)
Episode 23/100000 (22.0)
Episode 24/100000 (28.0)
Episode 25/100000 (14.0)
Episode 26/100000 (22.0)
Episode 27/100000 (15.0)
Episode 28/100000 (31.0)
Episode 29/100000 (66.0)
Episode 30/100000 (21.0)
Episode 31/100000 (23.0)
Episode 32/100000 (52.0)
Episode 33/100000 (48.0)
Episode 34/100000 (18.0)
Episode 35/100000 (74.0)
Episode 36/100000 (18.0)
Episode 37/100000 (31.0)
Episode 38/100000 (17.0)
Episode 39/100000 (16.0)
Episode 40/100000 (21.0)
Episode 41

In [0]:
np.shape(np.reshape([2,2],(-1,1)))

In [0]:
stats = q_learning(env, estimator, 100*1000, epsilon=0.01, epsilon_decay=1)
